Imported Libraries

In [1]:
import requests
import json
import time
import csv
import pandas as pd
import sklearn
import numpy as np
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer,HashingVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.cross_validation import train_test_split
from sklearn import linear_model
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier

/anaconda2/envs/dsi/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


We retrieve the information from reddit.com here. 

In [2]:
url = "http://www.reddit.com/r/nba.json"
url_2 = "http://www.reddit.com/r/nfl.json"

In [3]:
res = requests.get(url)
res = requests.get(url, headers={'User-agent': 'Vijay Bot 0.1'})
res2 = requests.get(url_2)
res2 = requests.get(url_2, headers={'User-agent': 'Vijay Bot 0.1'})

In [4]:
data = res.json()
data2 = res2.json()

The code below will read through the sub-reddit nba and download a series of nba reddit posts. 
Reddit will limit the number of requests per second you're allowed to make and for this reason and for this reason, it is important to end the code with time.sleep(3). 

In [5]:
all_posts = []
url = 'http://www.reddit.com/r/nba.json?after='
#each of these requests below does 26 and then 25 posts
for i in range(28):
    res = requests.get(url, headers={'User-agent': 'vijay'})
    data = res.json()
    for i in range(25):
        post = []
        post.append(data['data']['children'][i]['data']['title'])
        post.append(data['data']['children'][i]['data']['selftext'])
        post.append(data['data']['children'][i]['data']['subreddit'])
#     post.append(data['data']['children']['title'])
#     post.append(data['data']['children']['subreddit_name_prefixed'])
#     all_posts.append(post)
        all_posts.append(post)
    after = data['data']['after']
    url = 'http://www.reddit.com/r/nba.json?after=' + after
    
    #print('The current after: ', after)
    time.sleep(3)

The code below will read through the sub-reddit NFL and download a series of NFL reddit posts.

In [7]:
url = 'http://www.reddit.com/r/nfl.json?after='
#each of these requests below does 26 and then 25 posts
for i in range(28):
    res2 = requests.get(url, headers={'User-agent': 'vijay'})
    data2 = res2.json()
    for i in range(25):
        post = []
        post.append(data2['data']['children'][i]['data']['title'])
        post.append(data2['data']['children'][i]['data']['selftext'])
        post.append(data2['data']['children'][i]['data']['subreddit'])
#     post.append(data['data']['children']['title'])
#     post.append(data['data']['children']['subreddit_name_prefixed'])
#     all_posts.append(post)
        all_posts.append(post)
    after = data2['data']['after']
    url = 'http://www.reddit.com/r/nfl.json?after=' + after
    
    #print('The current after: ', after)
    time.sleep(3)

In [8]:
df = pd.DataFrame(all_posts, columns = ['Title','Text','Category'])
df.to_csv("nfl-nba.csv", sep='\t')

In [9]:
vectorizer = TfidfVectorizer(input='./scraped-data.csv', stop_words=sklearn.feature_extraction.text.ENGLISH_STOP_WORDS)
data = pd.read_csv('scraped-data-multinomial.csv',sep='\t')
data["Category"] = data["Category"].map(lambda x: 'nba' in x)*1

In [10]:
output = vectorizer.fit_transform(data.Text.values.astype('U'))

In [11]:
train_data_features = output.toarray()

Train-Test Split is a very important part of predicting unseen data. Train-test split allows us to divide the sample data into two categories: train and test. The data in the train category is the data that our model is fed to make its predictions. The data in the test category is the data our model is exposed to and is expected to make predictions from using the data the model was fed in the train data. In our case, we will be making predictions of what posts belong in the sub-reddit nba. 

In [12]:
## YOUR CODE HERE

x = train_data_features
y = data["Category"]
SEED = 2000
x_train, x_validation_and_test, y_train, y_validation_and_test = train_test_split(train_data_features, y, test_size=.02, random_state=SEED)
x_validation, x_test, y_validation, y_test = train_test_split(x_validation_and_test, y_validation_and_test, test_size=.5, random_state=SEED)

In [13]:
logistic = linear_model.LogisticRegression()
logistic.fit(x_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

### Accuracy

Accuracy = (True Positive + True Negative)/(Total Number of Values)

In [14]:
predicted = logistic.predict(x_train)
train_score = logistic.score(x_train, y_train)
score = logistic.score(x_test, y_test)
print(score)

0.8679245283018868


While the logistic regression model holds value in correctly predicting which category the reddit post belongs in, it is still important to test other statistical models. One such model would be the Random Forest Classification Model. The Random Forest Classification Model is an ensemble model that creates multiple trees using different combinations of features of the reddit posts such as the title or the reddit text content and at the end chooses the best performing tree. We want to compare the accuracy score of a random forest classification model with the logistic regression model.

In [15]:
## YOUR CODE HERE
param_grid = { 
    'n_estimators': [2, 7],
    'max_features': ['auto', 'sqrt', 'log2']
}
gs = GridSearchCV(RandomForestClassifier(), param_grid = param_grid)
gs.fit(x_train, y_train)
print(gs.best_score_)
print(gs.best_params_)

0.9173281703775411
{'max_features': 'sqrt', 'n_estimators': 7}


Looking at the accuracy scores of the logistic regression model and the random forest classifier, we can clearly see that the Random Forest model is better able to correctly predict the categories of the posts. In this case, the Random Forest model is better able to predict the posts that belong in the nba subreddit.